# Introduction to Agentic Workflows from Scratch

# 8 Levels of Agentic Workflows

1. Simple LLM Call
2. Chaining LLM calls
3. Routing llm calls
4. LLMs + functions in prompt
5. LLMs + structured outputs
6. LLMs + function calling
7. Agent loop
8. Agent as a step of an Agentic Workflow

# 1. Simple LLM Call

- [OpenAI API Quickstart](https://platform.openai.com/docs/quickstart)

In [137]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

In [138]:
from openai import OpenAI

client = OpenAI()

def level1_llm_call(prompt: str)->str:
    response = client.responses.create(
        model="gpt-5-mini",
        input=prompt
    )
    
    return response.output_text

level1_llm_call("What is an agentic workflow?")

'Short answer\nAn agentic workflow is a way of structuring a process so autonomous software agents — AI components that can plan, take actions, use tools, monitor outcomes, and adapt — carry out most of the steps needed to achieve higher-level goals, with humans intervening for oversight, exceptions, or final approval.\n\nWhat it means in practice\nInstead of a rigid, linear pipeline where each task is hand-coded, an agentic workflow gives an agent a goal and a set of capabilities (APIs, tools, knowledge, memory). The agent decomposes the goal into subtasks, chooses and calls tools, iterates based on feedback, and either completes the goal or escalates to a human. The agent is “agentic” because it acts autonomously and adapts rather than just executing a fixed sequence.\n\nCore components\n- Goal/specification: the high-level objective the agent must achieve.\n- Planner/Reasoner: decomposes goals into steps and generates plans.\n- Tools/Actions: APIs, database queries, web scrapers, co

# 2. Chaining LLM Calls

- https://www.anthropic.com/research/building-effective-agents
- https://github.com/anthropics/anthropic-cookbook/blob/main/patterns/agents/basic_workflows.ipynb

In [139]:
from typing import List

def llm_call(prompt: str) -> str:
    """
    Call an LLM with a prompt.
    """
    response = client.responses.create(
        model="gpt-5-mini",
        input=prompt
    )
    
    return response.output_text

def level2_chain_llm_calls(input: str, prompts: List[str]) -> str:
    """
    Chain LLM calls together.
    """
    result = input
    for i, prompt in enumerate(prompts):
        result = llm_call(f"{prompt}. Input: {result}")
        print(result)
        print("-" * 100)
    
    return result

input_topic = "Agentic Workflow"
prompts = [
    "Create a 3 bullet point essay plan for this topic",
    "Write this essay following the plan strictly",
]
level2_chain_llm_calls(input_topic, prompts)

- Define and scope "agentic workflow": introduce the concept as a workflow in which autonomous agents (software/AI or human+AI hybrids) pursue goals, make decisions, and coordinate actions within an environment; distinguish agentic from purely automated or scripted workflows; identify core components to analyze (agents, objectives, sensors/inputs, actuators/outputs, feedback loops, orchestration/coordination). Suggest concrete framing for the essay’s first section: formal definition, taxonomy (single vs. multi-agent; reactive vs. planning), and a brief illustrative example (e.g., an AI agent managing cloud resources or a multi-agent customer-support pipeline).

- Argue the benefits and practical use cases: show how agentic workflows increase scalability, adaptability, and efficiency by enabling decentralized decision-making, continuous optimization, and context-aware interventions; discuss productivity and personalization gains (DevOps automation, autonomous data pipelines, robotic pro

'1) Definition, scope, and core components\n\n- Formal definition\n  - An "agentic workflow" is a structured sequence of tasks and decisions carried out by one or more autonomous agents — either software/AI agents or human+AI hybrid agents — that pursue objectives, sense the environment, take actions, and coordinate to achieve goals in a production environment. Unlike purely automated or scripted workflows (which follow predetermined sequences and conditional branches), agentic workflows give agents authority to make decisions, plan, and adapt their behavior based on feedback and evolving context.\n\n- Distinguishing agentic from scripted automation\n  - Scripted automation: deterministic flows, explicit branching, human-specified rules; limited adaptivity.\n  - Agentic workflow: agents can reason, re-plan, trade off objectives, and coordinate dynamically; supports decentralized decision-making and emergent behaviors.\n\n- Core components to analyze\n  - Agents: autonomous decision-mak

# 3. Routing LLM Calls

In [140]:
from typing import Dict
routes = {
    "coding": """Take this coding problem and produce a solution in Python, 
    your output should be only the Python code and ONLY THAT.
    Here is the coding problem:
    """,
    "math": """
    Take this math problem and produce a solution for it, 
    Make sure to organize it nicely as an explanatory markdown tutorial,
    mixing markdown style text and latex for the math equations.
    Here is the math problem:
    """,
}

def level3_routing_llm_calls(input: str, routes: Dict[str, str]) -> str:
    """
    Route the input to the appropriate route based on the input.
    """
    selector_prompt = f"""
    Analyze the input and classify it as one of the following categories: {list(routes.keys())}
    Your output should ONLY be the category name and absolutely nothing else.
    For example:
    Input: 'Write a Python function to read a file'
    Output: 'coding'
    
    Input: 'Solve the equation 2x + 3 = 11'
    Output: 'math'
    
    Input: {input}""".strip()
    
    selector_response = llm_call(selector_prompt)
    print(f"Router output: {selector_response}")
    selected_prompt = routes[selector_response]
    print(f"Running the following prompt: {selected_prompt}")
    print("-" * 100)
    return llm_call(f"{selected_prompt}. Input: {input}")

In [141]:
output = level3_routing_llm_calls("Write some code that can load a pdf and extract the headers from the document.", routes)
print(output)

Router output: coding
Running the following prompt: Take this coding problem and produce a solution in Python, 
    your output should be only the Python code and ONLY THAT.
    Here is the coding problem:
    
----------------------------------------------------------------------------------------------------
#!/usr/bin/env python3
"""
Extract repeated page headers from a PDF.

Usage:
    python extract_pdf_headers.py /path/to/file.pdf

Requirements:
    pip install PyMuPDF
"""
import sys
import json
import re
from collections import defaultdict
try:
    import fitz  # PyMuPDF
except ImportError:
    sys.exit("PyMuPDF not installed. Install with: pip install PyMuPDF")

def normalize_text(s):
    s = s.strip()
    s = re.sub(r'\s+', ' ', s)
    # Remove common page number patterns like "Page 1", "Pg. 1", "- 1 -"
    s = re.sub(r'^(page|pg|p)\.?\s*\d+\b[:.\-–—]*\s*', '', s, flags=re.I)
    s = re.sub(r'\b(page|pg|p)\.?\s*\d+\b[:.\-–—]*\s*$', '', s, flags=re.I)
    s = re.sub(r'^[\-\u201

# 4. LLMs + functions in prompt

<img src="./2025-08-29-23-58-22.png" width=50%>

[Source](https://arxiv.org/pdf/2302.04761)

In [142]:
# pip install --upgrade openai
import os
import json
from pathlib import Path
from typing import Dict, Callable
import inspect

# --- Safe file helpers (restrict to a base directory) ---
BASE_DIR = Path.cwd()  # change if you want a sandbox like Path("./sandbox").resolve()

def _ensure_under_base(p: Path) -> Path:
    p = (BASE_DIR / p).resolve()
    if not str(p).startswith(str(BASE_DIR)):
        raise ValueError("Path escapes base directory")
    return p

def read_file(file_path: str) -> str:
    """Read UTF-8 text from a file under BASE_DIR."""
    p = _ensure_under_base(Path(file_path))
    return p.read_text(encoding="utf-8")

def write_file(file_path: str, content: str) -> str:
    """Write UTF-8 text to a file under BASE_DIR (create parents)."""
    p = _ensure_under_base(Path(file_path))
    p.parent.mkdir(parents=True, exist_ok=True)
    p.write_text(content, encoding="utf-8")
    return f"Wrote {len(content)} chars to {p.relative_to(BASE_DIR)}"


def level4_llm_and_functions_in_prompt(prompt: str, functions: List[Callable]):

    # This line gets the source code of the functions as plain strings
    function_info = " ".join([inspect.getsource(func) for func in functions])
    full_prompt_with_function_info = f"""
    Take this request from a user: {prompt}.
    If the request involves writing to a file or reading to a file,
    you can output a call to these functions which you have access to: 
    {function_info}.
    """
    response = client.responses.create(
        model="gpt-5-mini",
        input=full_prompt_with_function_info,
        instructions="You are a helpful assistant that can write and read files."
    )
    
    return response.output_text
    
output_function_call = level4_llm_and_functions_in_prompt("Write a file called 'test.txt' with the content 'Hello, world!'", [write_file, read_file])

In [143]:
print(output_function_call)

write_file("test.txt", "Hello, world!")


Now, to execute the functions we can use Python's built-in ['exec'](https://docs.python.org/3/library/functions.html#exec:~:text=This%20function%20supports,is%20None.) module.

In [144]:
exec(output_function_call)

In [145]:
!cat test.txt

Hello, world!

# 5. LLMs + Structured Outputs

- [OpenAI API docs structured outputs](https://platform.openai.com/docs/guides/structured-outputs)
- [JSON Schemas](https://json-schema.org/overview/what-is-jsonschema)
- [Jsonformer: A Bulletproof Way to Generate Structured JSON from Language Models.](https://github.com/1rgs/jsonformer)

There is evidence that the ability to produce structured outputs reliably is the result of:
- Specialized fine-tuning (Toolformer, OpenAI function-calling, Claude JSON mode) on tool-use / schema-call datasets.
- Decoding constraints (Jsonformer, constrained sampling, grammar-based decoding) to enforce syntax.

In [146]:
from pydantic import BaseModel, Field

class WriteFileOperation(BaseModel):
    file_path: str = Field(description="The path to the file to be written to or read from")
    content: str = Field(description="The content to be written to the file")

class ReadFileOperation(BaseModel):
    file_path: str = Field(description="The path to the file to be written to or read from")

prompt = "Write a file called 'test2.txt' with the content 'Hello, world Again!'"

response = client.responses.parse(
    model="gpt-5-mini",
    input=prompt,
    instructions="You are a helpful assistant that can write and read files.",
    text_format=WriteFileOperation)

output_write_file_ops = response.output_parsed
print(output_write_file_ops.file_path)
print(output_write_file_ops.content)

test2.txt
Hello, world Again!


In [147]:
response.output_parsed

WriteFileOperation(file_path='test2.txt', content='Hello, world Again!')

In [148]:
def level5_llm_structured(prompt):
    response = client.responses.parse(
        model="gpt-5-mini",
        input=prompt,
        instructions="You are a helpful assistant that can write and read files.",
        text_format=WriteFileOperation # Structured OUTPUT from the LLM!
        )
    
    output_args = response.output_parsed
    # Calling the function with the arguments obtained using structured outputs from the LLM.
    write_file(output_args.file_path, output_args.content)
    print("File was created!")
    return output_args

level5_llm_structured("Write a file called 'level5-structured-outputs.txt' with the content 'Level 5: Structured outputs!'")

File was created!


WriteFileOperation(file_path='level5-structured-outputs.txt', content='Level 5: Structured outputs!')

In [149]:
!cat level5-structured-outputs.txt

Level 5: Structured outputs!

# Level 6 - Function Calling

- https://platform.openai.com/docs/guides/function-calling

After a quick ChatGPT + Search chat, this is my current model of the relationship between structured outputs and function calling:

- [Function calling came first](https://openai.com/index/function-calling-and-other-api-updates/) (as a user-facing API feature).
- Structured outputs is the generalization of the same mechanism:
  - Instead of only producing function arguments, the model can produce any schema-constrained JSON.

Structured outputs as a technique is the foundation that makes function calling possible — but historically, the narrower [“function calling” was released first (as an API facing feature by OpenAI)](https://openai.com/index/function-calling-and-other-api-updates/).


- Conceptually: Structured outputs (schema adherence) underlie function calling.
- Chronologically: [Function calling was released first (June 2023)](https://openai.com/index/function-calling-and-other-api-updates/), [structured outputs generalized it later (2024)](https://openai.com/index/introducing-structured-outputs-in-the-api/).

In [150]:
from openai import OpenAI
import json

client = OpenAI()

# 1. Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "write_file",
        "description": "Write a file with the given content.",
        "parameters": {
            "type": "object",
            "properties": {
                "file_path": {
                    "type": "string",
                    "description": "The path to the file to write to.",
                },
                "content": {
                    "type": "string",
                    "description": "The content to write to the file.",
                },
            },
            "required": ["file_path", "content"],
        },
    },
    {
        "type": "function",
        "name": "read_file",
        "description": "Read a file with the given path.",
        "parameters": {
            "type": "object",
            "properties": {
                "file_path": {
                    "type": "string",
                    "description": "The path to the file to read.",
                },
            },
            "required": ["file_path"],
        },
    },
]

def read_file(file_path: str) -> str:
    """Read UTF-8 text from a file under BASE_DIR."""
    p = _ensure_under_base(Path(file_path))
    return p.read_text(encoding="utf-8")

def write_file(file_path: str, content: str) -> str:
    """Write UTF-8 text to a file under BASE_DIR (create parents)."""
    p = _ensure_under_base(Path(file_path))
    p.parent.mkdir(parents=True, exist_ok=True)
    p.write_text(content, encoding="utf-8")
    return f"Wrote {len(content)} chars to {p.relative_to(BASE_DIR)}"


# Create a running input list we will add to over time
input_list = [
    {"role": "user", "content": "Create a file called 'test-function-calling.txt' with the content 'Level 6: Function calling!'"}
]

# 2. Prompt the model with tools defined
response = client.responses.create(
    model="gpt-5-mini",
    tools=tools,
    tool_choice="auto", # setting the choice of tool call to be automatically selected by the model
    input=input_list,
)

# Save function call outputs for subsequent requests
input_list += response.output

for item in response.output:
    if item.type == "function_call":
        if item.name == "write_file":
            # 3. Execute the function logic
            func_args = json.loads(item.arguments)
            file_path = func_args["file_path"]
            content = func_args["content"]
            output_write_file = write_file(file_path, content)
            # 4. Provide function call results to the model
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "output_content": output_write_file
                })
            })
        elif item.name == "read_file":
            # 3. Execute the function logic
            # Here we can feed the item.arguments directly because it's only one argument
            output_read_file = read_file(json.loads(item.arguments))
            # 4. Provide function call results to the model
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "output_content": output_read_file
                })
            })

print("Final input:")
response = client.responses.create(
    model="gpt-5-mini",
    instructions="Answer the user's question leveraging the information obtained from the usage of the tools.",
    tools=tools,
    input=input_list,
)

# 5. The model should be able to give a response!
output_json = response.model_dump_json(indent=2)
print(output_json)
print("Final output:")
print("\n" + response.output_text)

Final input:
{
  "id": "resp_68b270c1441c81a1b86d32b2293d2a030751cf3d6695eb09",
  "created_at": 1756524737.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "Answer the user's question leveraging the information obtained from the usage of the tools.",
  "metadata": {},
  "model": "gpt-5-mini-2025-08-07",
  "object": "response",
  "output": [
    {
      "id": "msg_68b270c20cf081a1b710c059788ed1860751cf3d6695eb09",
      "content": [
        {
          "annotations": [],
          "text": "Created file 'test-function-calling.txt' with the requested content.",
          "type": "output_text",
          "logprobs": []
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [
    {
      "name": "write_file",
      "parameters": {
        "type": "object",
        "properties": {
          "file_path": {
            "type": "strin

In [151]:
!cat test-function-calling.txt

Level 6: Function calling!

# Level 7 - React style Agent Loop

- Paper: https://arxiv.org/pdf/2210.03629

<img src="./2025-08-30-00-06-29.png" width=60%>

In [154]:
import re

# ---------- Python-side tools ----------
def read_file(file_path: str) -> str:
    p = Path(file_path)
    if not p.exists():
        return f"Error: file not found: {file_path}"
    try:
        return p.read_text(encoding="utf-8")
    except Exception as e:
        return f"Error reading '{file_path}': {e}"

def write_file(file_path: str, content: str) -> str:
    p = Path(file_path)
    try:
        p.parent.mkdir(parents=True, exist_ok=True)
        p.write_text(content, encoding="utf-8")
        return f"Wrote {len(content)} chars to {file_path}"
    except Exception as e:
        return f"Error writing '{file_path}': {e}"

PY_TOOLS: Dict[str, Callable[..., str]] = {
    "read_file": read_file,
    "write_file": write_file,
}

# ---------- Responses API tool schema ----------
TOOLS = [
    {
        "type": "function",
        "name": "read_file",
        "description": "Read text content from a UTF-8 file relative to the working directory.",
        "parameters": {
            "type": "object",
            "properties": {
                "file_path": {
                    "type": "string",
                    "description": "Path to the file to read (e.g., 'data/input.txt')."
                }
            },
            "required": ["file_path"],
            "additionalProperties": False
        },
    },
    {
        "type": "function",
        "name": "write_file",
        "description": "Write UTF-8 text content to a file (creates directories if needed).",
        "parameters": {
            "type": "object",
            "properties": {
                "file_path": {
                    "type": "string",
                    "description": "Where to write (e.g., 'out/summary.txt')."
                },
                "content": {
                    "type": "string",
                    "description": "Text to write into the file."
                }
            },
            "required": ["file_path", "content"],
            "additionalProperties": False
        },
    },
]

instructions_prompt = """
    You are a careful assistant that can use tools to read and write files.
    - If a task involves files, call the appropriate tool with precise arguments.
    - After tool calls, summarize results for the user.
    - Only write files when explicitly asked or when necessary to complete the task.
    - If something is ambiguous (e.g., missing path), ask a concise clarifying question.
    When you are done, respond starting with 'Final Answer:' and keep it concise.
"""

def safe_load_json(s: str):
    try:
        return json.loads(s or "{}")
    except json.JSONDecodeError:
        s2 = (s or "").replace("'", '"').rstrip(",")
        try:
            return json.loads(s2)
        except Exception:
            return {}

def summarize_for_model(text: str, limit: int = 2000) -> str:
    t = str(text)
    return t if len(t) <= limit else (t[:limit] + f"... [truncated {len(t)-limit} chars]")

def level7_react_agent_loop(task_prompt: str, *, max_turns: int = 8) -> str:
    """
    ReAct-style loop using OpenAI Responses API with proper function_call / function_call_output items.
    """
    input_list = [
        {"role": "user", "content": task_prompt}
    ]
    tool_only_streak = 0

    for _ in range(max_turns):
        # 1) Ask the model with current transcript (items) + tools
        response = client.responses.create(
            model="gpt-5",
            tools=TOOLS,
            input=input_list,
            instructions=instructions_prompt,
        )

        # 2) Append all model output items to running input_list
        input_list += response.output

        # 3) Find any function calls in this turn
        fn_calls = [it for it in response.output if it.type == "function_call"]

        if fn_calls:
            tool_only_streak += 1
            # 4) Execute each function call and append function_call_output items
            for call in fn_calls:
                name = call.name
                args = safe_load_json(call.arguments)
                fn = PY_TOOLS.get(name)
                if not fn:
                    result = json.dumps({"error": f"unknown tool '{name}'", "available": list(PY_TOOLS)})
                else:
                    try:
                        result_text = fn(**args)
                    except TypeError as e:
                        result_text = f"TypeError for '{name}' with args {args}: {e}"
                    except Exception as e:
                        result_text = f"Tool '{name}' failed: {e}"
                    result = json.dumps({"result": summarize_for_model(result_text)})

                input_list.append({
                    "type": "function_call_output",
                    "call_id": call.call_id,
                    "output": result,
                })

            # 5) If the model is stuck in tool-only mode, nudge to conclude
            if tool_only_streak > 6:
                input_list.append({
                    "role": "user",
                    "content": "Please conclude with 'Final Answer:' now."
                })
            # Continue loop so the model can see the outputs we just appended
            continue

        # 6) No tool calls — try to return text
        tool_only_streak = 0
        text = (response.output_text or "").strip()
        if text:
            m = re.search(r"Final Answer:\s*(.*)", text, flags=re.I | re.S)
            return (m.group(1).strip() if m else text)

        # 7) Fallback nudge if we got non-text, non-function output
        input_list.append({"role": "user", "content": "Please provide a clear answer or call a tool."})

    return "Stopped: max_turns reached without a final answer."

# ---------- Demo ----------

user_task = (
    "Read 'sample_input.txt' and write a concise 2-sentence summary "
    "to 'summary_file.txt'. Then confirm what you did."
)
print("\n--- FINAL ---\n" + level7_react_agent_loop(user_task))


--- FINAL ---
- Read sample_input.txt and generated a concise 2-sentence summary.
- Wrote the summary to summary_file.txt.


# Level 8 - Agent as a Step of an Agentic Workflow

- [Building Effective Agents by Anthropic](https://www.anthropic.com/engineering/building-effective-agents)
- [Agents vs Workflows: Why Not Both? — Sam Bhagwat, Mastra.ai](https://youtu.be/8SUJEqQNClw?t=708)

<img src="2025-08-30-02-48-11.png" width=60%>

In [155]:
from pydantic import BaseModel, Field
from typing import Literal

class RouterOutput(BaseModel):
    category: Literal["joke","websearch"] = Field(description="The category of the input (joke or websearch).")

output = client.responses.parse(
    model="gpt-5-mini",
    input="I want to hear a joke",
    instructions="You are a helpful assistant that can tell jokes.",
    text_format=RouterOutput
)

print(output.output_parsed.category)

joke


In [ ]:
JOKER_AGENT_TOOLS = [
    {
        "type": "function",
        "name": "write_file",
        "description": "Write UTF-8 text content to a file (creates directories if needed).",
        "parameters": {
            "type": "object",
            "properties": {
                "file_path": {
                    "type": "string",
                    "description": "Where to write (e.g., 'out/summary.txt')."
                },
                "content": {
                    "type": "string",
                    "description": "Text to write into the file."
                }
            },
            "required": ["file_path", "content"],
            "additionalProperties": False
        },
    },
]

WEBSEARCH_AGENT_TOOLS = [
    {
        "type": "function",
        "name": "write_file",
        "description": "Write UTF-8 text content to a file (creates directories if needed).",
        "parameters": {
            "type": "object",
            "properties": {
                "file_path": {
                    "type": "string",
                    "description": "Where to write (e.g., 'out/summary.txt')."
                },
                "content": {
                    "type": "string",
                    "description": "Text to write into the file."
                }
            },
            "required": ["file_path", "content"],
            "additionalProperties": False
        },
    },
    {"type": "web_search"}
]

agent_routes = {
    "joke": {
        "name": "Joker Agent",
        "instructions": "You are an expert joke teller. Your output is always a super funny joke.",
        "tools": JOKER_AGENT_TOOLS
    },
    
    "websearch": {
        "name": "Web Search Agent",
        "instructions": "You are an expert web searcher. You use the web search tool to search \
            the web for the relevant information, given the user's input and then you output that\
            into an organized markdown style report saving it with the write_file tool.",
        "tools": WEBSEARCH_AGENT_TOOLS
    }
}

In [159]:
# The web search tool is called automatically by the OpenAI's responses
tools_map = {
    "write_file": write_file,
}

In [160]:
def react_agent_loop(input: str, agent: Dict[str, str], *, max_turns: int = 3) -> str:
    """
    ReAct-style loop using OpenAI Responses API with proper function_call / function_call_output items.
    """
    input_list = [
        {"role": "user", "content": input}
    ]
    tool_only_streak = 0
    AGENT_TOOLS = agent["tools"]

    for i in range(max_turns):
        print("--------------------------------")
        print(f"Iteration: {i}")
        # 1) Ask the model with current transcript (items) + tools
        response = client.responses.create(
            model="gpt-5-mini",
            tools=AGENT_TOOLS,
            input=input_list,
            instructions=agent["instructions"],
        )
        

        # 2) Append all model output items to running input_list
        input_list += response.output

        # 3) Find any function calls in this turn
        fn_calls = [it for it in response.output if it.type == "function_call"]

        if fn_calls:
            tool_only_streak += 1
            # 4) Execute each function call and append function_call_output items
            for call in fn_calls:
                name = call.name
                args = safe_load_json(call.arguments)
                fn = tools_map.get(name)
                if not fn:
                    result = json.dumps({"error": f"unknown tool '{name}'", "available": AGENT_TOOLS})
                else:
                    try:
                        result_text = fn(**args)
                    except TypeError as e:
                        result_text = f"TypeError for '{name}' with args {args}: {e}"
                    except Exception as e:
                        result_text = f"Tool '{name}' failed: {e}"
                    result = json.dumps({"result": summarize_for_model(result_text)})

                input_list.append({
                    "type": "function_call_output",
                    "call_id": call.call_id,
                    "output": result,
                })

            # 5) If the model is stuck in tool-only mode, nudge to conclude
            if tool_only_streak > 6:
                input_list.append({
                    "role": "user",
                    "content": "Please conclude with 'Final Answer:' now."
                })
            # Continue loop so the model can see the outputs we just appended
            continue

        # 6) No tool calls — try to return text
        tool_only_streak = 0
        text = (response.output_text or "").strip()
        if text:
            m = re.search(r"Final Answer:\s*(.*)", text, flags=re.I | re.S)
            return (m.group(1).strip() if m else text)

        # 7) Fallback nudge if we got non-text, non-function output
        input_list.append({"role": "user", "content": "Please provide a clear answer or call a tool."})

    return "Stopped: max_turns reached without a final answer."

In [161]:
def level8_agent_as_step_of_agentic_workflow(input: str, agent_routes: Dict[str, str]) -> str:
    """
    Route the input to the appropriate route based on the input.
    """
    
    output = client.responses.parse(
        model="gpt-5-mini",
        input=input,
        instructions="You are a helpful assistant that can tell jokes.",
        text_format=RouterOutput)
    print(f"Router output: {output.output_parsed.category}")
    selected_agent = agent_routes[output.output_parsed.category]
    print(f"Selected agent: {selected_agent['name']}")
    print("-" * 100)
    return react_agent_loop(input, selected_agent)

In [162]:
input_joke = "Tell me a joke about artificial intelligence."
level8_agent_as_step_of_agentic_workflow(input_joke, agent_routes)

Router output: joke
Selected agent: Joker Agent
----------------------------------------------------------------------------------------------------
--------------------------------
Iteration: 0


'Why did the AI go to art school? To learn how to draw better conclusions.'

In [163]:
input_joke = "Write 5 funny jokes about over hyping AI and save them to a file called 'ai-jokes.md'"
level8_agent_as_step_of_agentic_workflow(input_joke, agent_routes)

Router output: joke
Selected agent: Joker Agent
----------------------------------------------------------------------------------------------------
--------------------------------
Iteration: 0
--------------------------------
Iteration: 1


'Saved to ai-jokes.md.\n\nHere are the 5 jokes about overhyping AI:\n\n1. They said AI would replace my job, so I taught it to make coffee. Now it runs a hugely successful TikTok about latte art while I keep getting promoted for actually doing the work.\n\n2. The hype said AI would make everything smarter. Turns out my toaster only got smarter at asking for a software update — every time it burns the toast it blames "legacy carbs."\n\n3. Investors poured money into an AI that "understands human emotion." It nodded thoughtfully at meetings, charged premium subscription fees, and ghosted everyone when the project went dark.\n\n4. I asked an AI for life advice and it replied, "According to my analysis, you should buy Bitcoin and start a podcast." I asked it for dinner ideas and it suggested starting a podcast about Bitcoin instead.\n\n5. The company branded its chatbot as "groundbreaking" and "revolutionary." It was mainly used to forward office memes and pretend to be in meetings. Revolu

In [164]:
input_websearch = """
Research the latest papers from 2025 on building agentic 
workflows and write me a summary of what you found to a 
file called 'agentic_workflows_2025_summary.md'
"""
level8_agent_as_step_of_agentic_workflow(input_websearch, agent_routes)

Router output: websearch
Selected agent: Web Search Agent
----------------------------------------------------------------------------------------------------
--------------------------------
Iteration: 0
--------------------------------
Iteration: 1


"I've saved the summary to agentic_workflows_2025_summary.md (on the assistant's file system). The file contains a concise 2025-focused literature review, key papers, trends, open challenges, and recommendations, with citations to the sources I found.\n\nWould you like me to:\n- Expand any paper's entry into a longer annotated note with key figures/method details?\n- Fetch and attach the PDFs or GitHub repos for selected papers (AFlow, MermaidFlow, etc.)?\n- Produce slides or a short presentation summarizing the findings for a team?"